## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [23]:
import mne
import numpy as np

file_path = "datasets/avi/single/sub1_combined-raw.fif"

epochs = mne.read_epochs(file_path, preload=True)

threshold = 0.25


# stimulated_frequencies = [6.0, 6.5, 7.0, 7.5, 8.2, 9.3,10.0]  
psd_values = []

targets = [float(item) for item in epochs.event_id.keys()]

features = list()
## passo 3: computar PSD para cada frequencia


for i in range(len(epochs)):
    sample = list()
    for freq in targets:
        # filtra os dados para a faixa de frequencia 
        fmin = freq - threshold
        fmax = freq + threshold
        sample.append(epochs[i].compute_psd(fmin=fmin, fmax=fmax,verbose=False)[0][0])
    features.append(sample)

X = np.array(features)
# passo 4: extrair recursos dos valores PSD
X = X.reshape((X.shape[0], X.shape[1], X.shape[-1]))

max_values = np.max(X, axis=-1)
max_values.shape

#passo 5: classificar os vetores de recursos
file_path = "datasets/avi/single/labels_sub1.npy"
y = np.load(file_path).flatten()
print(y, y.shape)


i_max = np.argmax(max_values, axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print("Acuracia: {}%".format(acc))

# print("vals previstos: {}".format(predictions))
# print("frequencias: {}".format(targets))


Reading c:\Users\caio-\Documents\patter-recog\SSVEP\datasets\avi\single\sub1_combined-raw.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   29998.05 ms
        0 CTF compensation matrices available
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated


C:\Users\caio-\AppData\Local\Temp\ipykernel_6192\322334082.py:6: RuntimeWarning: This filename (datasets/avi/single/sub1_combined-raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file_path, preload=True)


[10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3
 10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3] (84,)
